# Find your soulmate

I created this little notebook during a Data Science bootcamp. One day we were jokingly asking each other "light-hearted" preference questions like 'are you a cat or dog person' or 'pineapple on pizza yes/no' and I decided to set up a small google survey and wrote this notebook so that we could find out our bootcamp soulmates. 

Calculates the number of matching answers between participants and adds the most similar participant(s) as last column.  
Download results of survey as data.csv

Suspected issue:
The script might not work if two Survey participants have the same name.

In [ ]:
# libraries
import pandas as pd
import numpy as np

import holoviews as hv
from holoviews import opts, dim
from bokeh.sampledata.les_mis import data
from bokeh.io import export_png

In [ ]:
df = pd.read_csv("data/data.csv")

In [ ]:
df = df.drop("Timestamp",errors="ignore",axis=1)
col = df.pop("Name:")
df.insert(0,"name",col) # Name as first column

In [ ]:
df = df.set_index(df.name)

In [ ]:
zero_arr = np.zeros((df.shape[0],df.shape[0]), int) # Get a n x n matrix where n = number of participants in the survey

mat = pd.DataFrame(zero_arr, columns=df.name,index=df.name) # Create the n x n matrix as df with names' as columns.


In [ ]:
for name1 in df.name:
    matches = []
    for name2 in df.name:
        matches.append(sum(df.loc[name1] == df.loc[name2]))
    mat.loc[name1] = matches  # Counts number of matches row by row.

In [ ]:
np.fill_diagonal(mat.values,0) # Set diagonal to 0 so that max() works.

In [ ]:
# mat.loc["Jan"]

In [ ]:
mat.columns[mat.loc["Jan"] == mat.loc["Jan"].max()]

In [ ]:
max_li = []
for name in mat.columns:
    mat.loc[name] == mat.loc[name].max()
    max_li.append(mat.columns[mat.loc[name] == mat.loc[name].max()].tolist()) # Selects those columns(=names) with Max value in row.
mat["best_match"] = max_li

# Loop instead of idmax() so that ties can be seen.

In [ ]:
mat.to_csv("best-team.csv")

In [ ]:
for c in df.columns:
    print("---- %s ---" % c)
    print(df[c].value_counts())

In [ ]:
# Sample heatmap-like data (replace this with your heatmap data)
df = mat.reset_index().drop("best_match", axis=1).set_index("name")
# Initialize empty lists to store person1, person2, and value
person1_list = []
person2_list = []
value_list = []

# Extract non-zero values from the upper triangle of the heatmap
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        person1 = df.columns[i]
        person2 = df.columns[j]
        value = df.iloc[i, j]
        
        # Append the values to the respective lists
        person1_list.append(person1)
        person2_list.append(person2)
        value_list.append(value)

# Create a new DataFrame in the person1, person2, value format
result_df = pd.DataFrame({
    'person1': person1_list,
    'person2': person2_list,
    'value': value_list
})




In [ ]:
# friendly adapted code from  https://python-graph-gallery.com/231-chord-diagram-with-bokeh/


# Should the diagram be plotted with 'bokeh' or 'matplotlib'?
hv.extension('bokeh')
# How large should be the diagram?
hv.output(size=300)
# nodes = hv.Dataset(pd.DataFrame(result_df['person1']), 'index')
chord = hv.Chord((result_df)).select(value=(12, None))
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('person1').str(), 
               labels='person1', node_color=dim('index').str()))

# Not needed in a jupyter notebook
# It shows the diagram when run in another IDE or from a python script:
from bokeh.plotting import show
show(hv.render(chord))


In [ ]:
#export_png(hv.render(chord), filename="soulmates-chord.png")